### Imports

In [1]:
# import public packages
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys


from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram, hann, butter, filtfilt
from scipy import signal, interpolate
from scipy.interpolate import make_interp_spline, BSpline
from datetime import datetime
from io import open
from importlib import reload

# import own functions
from utils import find_folders

### Add Directories / Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
ftg_path = find_folders.get_onedrive_path("FTG")

In [ ]:
raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        ftg_path,
        "Sub021",
        "perceive_output",
        "sub-021_ses-2022082612233578_run-BrainSense20220826125900.mat"
    ),
    info = None
)

sub = 'Sub021'

taskdict = {
    'RampUpThres': 1,
    'FastRamp': 2,
    '110Hz': 3,
    'Bilateral': 4,
    '145Hz': 5
}

#### Artefact Rejection & FIF Files

In [ ]:
#Interactive plot for artefact rejection
fig = raw.plot(n_channels = 2, highpass = 5, lowpass = 100, 
    filtorder = 5, duration = 20)
fig.fake_keypress('a')

In [ ]:
#First time making them
interactive_annot = raw.annotations
raw.annotations.save('Sub033_RampUpThres_artefactsAnnotations.csv', 
    overwrite = True)

In [ ]:
#Importing them later
interactive_annot = pd.read_csv(
    os.path.join(
        ftg_path,
        'Rec_Annotations',
        'Sub021_Freq110Hz_artefactsAnnotations.csv'
    )
)

#fix timing in imported annotations
new_onsets = fix_annot_onsets(interactive_annot)
print(new_onsets)

In [ ]:
my_annot = mne.Annotations(onset=new_onsets,  # in seconds
                           duration=annot_from_file.duration,  # in seconds, too
                           description=annot_from_file.description,
                           orig_time=raw.info['meas_date'])